<a href="https://colab.research.google.com/github/germancruzram/Frame-marker/blob/main/Frame_maker_shared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PYTHON PARA UN PROYECTO VIAL♠**  

Ing. German Cruz R.🇳


# **1. INSTALANDO LAS LIBRERIAS A UTILIZAR**

In [5]:
!pip install exifread
!pip install piexif
!!pip install geopy
!pip install rarfile
!pip install pillow
!pip install Pillow piexif
!pip install gdown
!sudo apt-get update
!sudo apt-get install unrar -y

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


# **2.CONFIGURANDO CARPETAS DE TRABAJO**

In [6]:
import os


folder_paths = ["/content/imagenes", "/content/imagenes_con_telemetria"]

for folder_path in folder_paths:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"La carpeta {folder_path} ha sido creada con éxito.")
    else:
        print(f"La carpeta {folder_path} ya existe.")
    !ls


La carpeta /content/imagenes ya existe.
imagenes  imagenes_con_telemetria  sample_data
La carpeta /content/imagenes_con_telemetria ya existe.
imagenes  imagenes_con_telemetria  sample_data


**RECUERDE CARGAR LAS IMAGENES EN LA CARPETA "IMAGENES"**

**INDIQUE LA CARPETA EN DRIVE PARA IMPORTARLA"**

In [16]:
# Importar la biblioteca necesaria
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import shutil
import os

# Ruta de origen en Google Drive
source_folder = '/content/drive/My Drive/100'

# Ruta de destino en el entorno de Colab
destination_folder = '/content/imagenes'

# Crear el directorio de destino si no existe
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Copiar todos los archivos y carpetas del directorio de origen al de destino
for item in os.listdir(source_folder):
    source_path = os.path.join(source_folder, item)
    destination_path = os.path.join(destination_folder, item)

    if os.path.isdir(source_path):
        # Copiar directorio
        shutil.copytree(source_path, destination_path)
    else:
        # Copiar archivo
        shutil.copy2(source_path, destination_path)

print(f'Contenido de {source_folder} copiado a {destination_folder}')


Contenido de /content/drive/My Drive/100 copiado a /content/imagenes


# **3.COMPRIMIENDO IMAGENES**

In [ ]:
import os
from PIL import Image
from concurrent.futures import ThreadPoolExecutor


image_folder = '/content/imagenes'

new_resolution = (1280, 1024)

def resize_image_with_exif(image_path, new_resolution):
    try:
        with Image.open(image_path) as img:
            exif = img.info.get('exif')
            img = img.resize(new_resolution, Image.LANCZOS)
            img.save(image_path, exif=exif)
        print(f"Redimensionado {image_path}")
    except Exception as e:
        print(f"Error redimensionando {image_path}: {e}")

def resize_images_in_folder(image_folder, new_resolution):
    image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.lower().endswith(('png', 'jpg', 'jpeg', 'tiff', 'bmp', 'gif'))]

    original_filenames = set(image_files)

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(resize_image_with_exif, image_path, new_resolution) for image_path in image_files]
        for future in futures:
            future.result()

    resized_filenames = set(os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.lower().endswith(('png', 'jpg', 'jpeg', 'tiff', 'bmp', 'gif')))

    if original_filenames == resized_filenames:
        print("Nombres de archivos consistentes después de redimensionar")
    else:
        print("Advertencia: Los nombres de los archivos han cambiado")

resize_images_in_folder(image_folder, new_resolution)
print("Imágenes redimensionadas y guardadas en la carpeta", image_folder)

# **4.ORDENANDO LAS FOTOS SECUENCIALMENTE**

In [ ]:
import os
import piexif
import exifread

def get_image_datetime(image_path):
    """
    Obtiene la fecha y hora de captura de una imagen utilizando la información Exif.
    Args:
        image_path (str): La ruta de la imagen.
    Returns:
        str: La fecha y hora de captura en formato de cadena si está disponible,
             una fecha mínima si no hay información EXIF.
    """
    try:
        exif_data = piexif.load(image_path)
        date_time_original = exif_data["Exif"][piexif.ExifIFD.DateTimeOriginal]
        return date_time_original.decode('utf-8')
    except (KeyError, AttributeError, ValueError):
        return "0000:00:00 00:00:00"  # Fecha mínima si no hay información EXIF

def get_coordinates(image_path):
    """
    Obtiene las coordenadas GPS de una imagen utilizando la información Exif.
    Args:
        image_path (str): La ruta de la imagen.
    Returns:
        tuple: Una tupla con latitud y longitud en formato decimal si están disponibles,
               None si no hay información EXIF.
    """
    try:
        with open(image_path, 'rb') as f:
            exif_tags = exifread.process_file(f)
            if 'GPS GPSLatitude' in exif_tags and 'GPS GPSLongitude' in exif_tags:
                lat_deg = float(exif_tags['GPS GPSLatitude'].values[0].num) / float(exif_tags['GPS GPSLatitude'].values[0].den)
                lat_min = float(exif_tags['GPS GPSLatitude'].values[1].num) / float(exif_tags['GPS GPSLatitude'].values[1].den)
                lat_sec = float(exif_tags['GPS GPSLatitude'].values[2].num) / float(exif_tags['GPS GPSLatitude'].values[2].den)
                latitude_dec = lat_deg + (lat_min / 60.0) + (lat_sec / 3600.0)
                if exif_tags['GPS GPSLatitudeRef'].printable == 'S':
                    latitude_dec = -latitude_dec

                lon_deg = float(exif_tags['GPS GPSLongitude'].values[0].num) / float(exif_tags['GPS GPSLongitude'].values[0].den)
                lon_min = float(exif_tags['GPS GPSLongitude'].values[1].num) / float(exif_tags['GPS GPSLongitude'].values[1].den)
                lon_sec = float(exif_tags['GPS GPSLongitude'].values[2].num) / float(exif_tags['GPS GPSLongitude'].values[2].den)
                longitude_dec = lon_deg + (lon_min / 60.0) + (lon_sec / 3600.0)
                if exif_tags['GPS GPSLongitudeRef'].printable == 'W':
                    longitude_dec = -longitude_dec

                return latitude_dec, longitude_dec
    except Exception as e:
        print(f"Error obteniendo coordenadas de {image_path}: {e}")
    return None

def process_images_by_coordinates(folder_path):
    """
    Procesa las imágenes en la carpeta especificada basándose en las coordenadas GPS.
    Las imágenes se procesan de forma consecutiva según las coordenadas GPS, desde la primera a la última en la trayectoria.
    Args:
        folder_path (str): La ruta de la carpeta que contiene las imágenes.
    """
    files = os.listdir(folder_path)
    image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Obtener las coordenadas y las fechas de captura de las imágenes
    image_data = []
    for image_name in image_files:
        image_path = os.path.join(folder_path, image_name)
        coordinates = get_coordinates(image_path)
        datetime = get_image_datetime(image_path)
        if coordinates:
            image_data.append((image_name, coordinates, datetime))

    # Ordenar las imágenes por las coordenadas y luego por la fecha y hora de captura
    image_data.sort(key=lambda x: (x[1], x[2]))

    # Procesar las imágenes en el orden correcto
    for image_name, _, _ in image_data:
        image_path = os.path.join(folder_path, image_name)
        # Aquí puedes agregar el procesamiento que necesites hacer con cada imagen
        print(f"Procesando {image_name} con coordenadas {_}")

folder_path = "/content/imagenes"

process_images_by_coordinates(folder_path)



# **5.LISTANDO IMAGENES CON COORDENADAS Y CALCULO DE DISTANCIAS 3D**

In [20]:
import os
import exifread
import pandas as pd
import math

def get_decimal_degrees(dms, ref):
    degrees = dms.values[0].num / dms.values[0].den
    minutes = dms.values[1].num / dms.values[1].den
    seconds = dms.values[2].num / dms.values[2].den
    decimal = degrees + (minutes / 60.0) + (seconds / 3600.0)
    if ref in ['S', 'W']:
        decimal *= -1
    return decimal

def extract_image_info(image_path, prev_coordinates=None, accumulated_distance=0):
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f)
        if not tags:
            return None, None, None, None, None, 0, accumulated_distance

        latitude = tags.get('GPS GPSLatitude')
        longitude = tags.get('GPS GPSLongitude')
        latitude_ref = tags.get('GPS GPSLatitudeRef')
        longitude_ref = tags.get('GPS GPSLongitudeRef')
        altitude = tags.get('GPS GPSAltitude')
        date_time = tags.get('EXIF DateTimeOriginal')

        if latitude and longitude and latitude_ref and longitude_ref:
            latitude_dec = get_decimal_degrees(latitude, latitude_ref.printable)
            longitude_dec = get_decimal_degrees(longitude, longitude_ref.printable)
        else:
            latitude_dec, longitude_dec = None, None

        altitude_val = altitude.values[0].num / altitude.values[0].den if altitude else None
        date_time_str = str(date_time) if date_time else None

        distance = 0
        if prev_coordinates and latitude_dec is not None and longitude_dec is not None:
            if altitude_val is not None:

                lat1 = math.radians(prev_coordinates[0])
                lon1 = math.radians(prev_coordinates[1])
                lat2 = math.radians(latitude_dec)
                lon2 = math.radians(longitude_dec)

                R = 6371.0 * 1000
                d_altitude = abs(altitude_val - prev_coordinates[2])
                dlat = lat2 - lat1
                dlon = lon2 - lon1

                a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
                c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
                distance_horizontal = R * c
                distance = math.sqrt(distance_horizontal ** 2 + d_altitude ** 2) / 1000  # Convertir a kilómetros

        accumulated_distance += distance

        return (os.path.basename(image_path), latitude_dec, longitude_dec, altitude_val, date_time_str, distance, round(accumulated_distance, 4))

def create_excel_with_image_info(folder_path, excel_path, initial_accumulated_distance=0):
    image_info_list = []
    prev_coordinates = None
    accumulated_distance = initial_accumulated_distance

    for filename in sorted(os.listdir(folder_path)):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(folder_path, filename)
            info = extract_image_info(image_path, prev_coordinates, accumulated_distance)
            if info[1] is not None and info[2] is not None:
                prev_coordinates = (info[1], info[2], info[3])
                accumulated_distance = info[6]
            image_info_list.append(info)

    df = pd.DataFrame(image_info_list, columns=['Nombre del Archivo', 'Latitud', 'Longitud', 'Altitud', 'Fecha y Hora', 'Distancia (km)', 'Distancia Acumulada (km)'])
    df.sort_values(by=['Fecha y Hora'], inplace=True)
    df.to_excel(excel_path, index=False)
    print("Excel generado con éxito.")


folder_path = "/content/imagenes"
excel_path = "/content/informacion_imagenes.xlsx"
initial_accumulated_distance = float(input("Ingrese el valor inicial de distancia acumulada: "))
create_excel_with_image_info(folder_path, excel_path, initial_accumulated_distance)

!ls

/usr/local/lib/python3.10/dist-packages/pandas/core/arrays/masked.py:64: RuntimeWarning: coroutine 'main' was never awaited
  from pandas.core import (


Ingrese el valor inicial de distancia acumulada: 0
Excel generado con éxito.
drive  imagenes  imagenes_con_telemetria  informacion_imagenes.xlsx  sample_data


# **6.MARCADO DE FOTOGRAFIAS** ✏

1.   Coordenadas geográficas
2.   Fecha y hora de captura
3.   Referencia kilométrica






In [21]:
import os
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime
import exifread
import zipfile
import piexif
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Ruta de la carpeta con las imágenes
folder_path = "/content/imagenes"
output_folder_path = "/content/imagenes_con_telemetria"
excel_path = "/content/informacion_imagenes.xlsx"
zip_output_path = '/content/imagenes_con_telemetria.zip'

# Crear carpeta de salida si no existe
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Cargar datos del archivo Excel
df = pd.read_excel(excel_path)

# Rutas de las fuentes
font_path = "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf"
font_size_coords = 30
font_size_datetime = 30
font_size_distance = 40
text_color = (255, 255, 255)

# Función para obtener coordenadas GPS
def get_coordinates(image_path):
    try:
        with open(image_path, 'rb') as f:
            exif_tags = exifread.process_file(f)
            if 'GPS GPSLatitude' in exif_tags and 'GPS GPSLongitude' in exif_tags:
                lat_deg = float(exif_tags['GPS GPSLatitude'].values[0].num) / float(exif_tags['GPS GPSLatitude'].values[0].den)
                lat_min = float(exif_tags['GPS GPSLatitude'].values[1].num) / float(exif_tags['GPS GPSLatitude'].values[1].den)
                lat_sec = float(exif_tags['GPS GPSLatitude'].values[2].num) / float(exif_tags['GPS GPSLatitude'].values[2].den)
                latitude_dec = lat_deg + (lat_min / 60.0) + (lat_sec / 3600.0)
                lon_deg = float(exif_tags['GPS GPSLongitude'].values[0].num) / float(exif_tags['GPS GPSLongitude'].values[0].den)
                lon_min = float(exif_tags['GPS GPSLongitude'].values[1].num) / float(exif_tags['GPS GPSLongitude'].values[1].den)
                lon_sec = float(exif_tags['GPS GPSLongitude'].values[2].num) / float(exif_tags['GPS GPSLongitude'].values[2].den)
                longitude_dec = lon_deg + (lon_min / 60.0) + (lon_sec / 3600.0)
                lat_dir = exif_tags['GPS GPSLatitudeRef'].printable
                lon_dir = exif_tags['GPS GPSLongitudeRef'].printable
                return latitude_dec, lat_dir, longitude_dec, lon_dir
    except Exception as e:
        print(f"Error obteniendo coordenadas de {image_path}: {e}")
    return None

# Función para agregar telemetría a la imagen
def add_telemetry(image_path, df):
    try:
        img = Image.open(image_path)
        draw = ImageDraw.Draw(img)
        coordinates = get_coordinates(image_path)

        # Añadir coordenadas GPS
        if coordinates:
            latitude_dec, lat_dir, longitude_dec, lon_dir = coordinates
            font_coords = ImageFont.truetype(font_path, size=font_size_coords)
            latitud = f"Lat: {latitude_dec:.5f} {lat_dir}"
            longitud = f"Long: {longitude_dec:.5f} {lon_dir}"
            position_lat = (50, img.height - font_size_coords*2.5)
            position_long = (50, img.height - font_size_coords*1.35)
            draw.text(position_lat, latitud, fill=text_color, font=font_coords)
            draw.text(position_long, longitud, fill=text_color, font=font_coords)

        # Añadir fecha y hora de captura
        exif_data = img._getexif()
        if exif_data and 36867 in exif_data:
            capture_datetime = exif_data[36867]
            capture_datetime = datetime.strptime(capture_datetime, "%Y:%m:%d %H:%M:%S")
            capture_datetime_str = capture_datetime.strftime("%Y-%m-%d %H:%M:%S")
            font_datetime = ImageFont.truetype(font_path, size=font_size_datetime)
            text_bbox = draw.textbbox((0, 0), capture_datetime_str, font=font_datetime)
            position_datetime = (img.width - text_bbox[2] - 50, img.height - text_bbox[3] - 50)
            draw.text(position_datetime, capture_datetime_str, fill=text_color, font=font_datetime)

        # Añadir distancia acumulada
        font_distance = ImageFont.truetype(font_path, size=font_size_distance)
        distance = df[df['Nombre del Archivo'] == os.path.basename(image_path)]['Distancia Acumulada (km)'].iloc[0]
        integer_part = int(distance)
        decimal_part = int((distance - integer_part) * 1000)
        distance_str = f"PKM {integer_part}+{decimal_part:03d}"
        text_bbox = draw.textbbox((0, 0), distance_str, font=font_distance)
        position_distance = ((img.width - text_bbox[2]) // 2, img.height - text_bbox[3] - 20)
        draw.text(position_distance, distance_str, fill=text_color, font=font_distance)

        # Guardar la imagen con la telemetría agregada
        output_path = os.path.join(output_folder_path, os.path.basename(image_path))
        img.save(output_path, exif=img.info.get("exif", b""))
        return output_path
    except Exception as e:
        print(f"Error procesando {image_path}: {e}")
        return None

# Procesar imágenes en paralelo
def process_images_parallel(folder_path, df):
    image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(lambda image_path: add_telemetry(image_path, df), image_files))
    return [result for result in results if result is not None]

# Ejecutar el procesamiento de las imágenes
processed_images = process_images_parallel(folder_path, df)

# Crear un archivo ZIP con las imágenes procesadas
with zipfile.ZipFile(zip_output_path, 'w') as zipf:
    for image_path in processed_images:
        zipf.write(image_path, arcname=os.path.basename(image_path))

print(f"Imágenes procesadas y guardadas en {zip_output_path}")



Imágenes procesadas y guardadas en /content/imagenes_con_telemetria.zip


In [22]:
import os
from zipfile import ZipFile

# Ruta de la carpeta a comprimir
folder_path = '/content/imagenes'

# Nombre del archivo ZIP
zip_filename = 'imagenes.zip'

# Crear el archivo ZIP
with ZipFile(zip_filename, 'w') as zip_file:
    # Recorrer todos los archivos y carpetas dentro de la carpeta
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # Construir la ruta completa del archivo
            file_path = os.path.join(root, file)
            # Agregar el archivo al archivo ZIP
            zip_file.write(file_path)

print(f'La carpeta "{folder_path}" se ha comprimido en el archivo "{zip_filename}".')

# Descargar el archivo ZIP
from google.colab import files
files.download(zip_filename)


La carpeta "/content/imagenes" se ha comprimido en el archivo "imagenes.zip".


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **7.GENERAR UN MAPA GIS CON LAS IMAGENES ETIQUETADAS**

In [ ]:
import os
import exifread
import folium
from folium import IFrame
import base64
import pandas as pd
from IPython.display import display, HTML

def get_coordinates(tags):
    latitude = tags.get('GPS GPSLatitude')
    longitude = tags.get('GPS GPSLongitude')
    if latitude and longitude:
        lat_deg = latitude.values[0].num / latitude.values[0].den
        lat_min = latitude.values[1].num / latitude.values[1].den
        lat_sec = latitude.values[2].num / latitude.values[2].den
        lat_ref = tags.get('GPS GPSLatitudeRef').values
        latitude = lat_deg + lat_min / 60.0 + lat_sec / 3600.0
        if lat_ref == 'S':
            latitude *= -1

        lon_deg = longitude.values[0].num / longitude.values[0].den
        lon_min = longitude.values[1].num / longitude.values[1].den
        lon_sec = longitude.values[2].num / longitude.values[2].den
        lon_ref = tags.get('GPS GPSLongitudeRef').values
        longitude = lon_deg + lon_min / 60.0 + lon_sec / 3600.0
        if lon_ref == 'W':
            longitude *= -1

        return latitude, longitude
    return None

def add_image_marker(map_obj, image_path, popup_text):
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f)
        coordinates = get_coordinates(tags)
        if coordinates:
            latitude, longitude = coordinates

            with open(image_path, "rb") as image_file:
                encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

            popup_html = f'<img src="data:image/jpeg;base64,{encoded_string}" width="500px"><br>{popup_text}'
            popup = folium.Popup(IFrame(html=popup_html, width=480, height=360), max_width=2650)
            folium.Marker(location=[latitude, longitude], popup=popup).add_to(map_obj)

def display_images_on_map(folder_path, df):
    coordinates = []
    image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(('jpg', 'jpeg', 'png'))]

    for image_path in image_paths:
        with open(image_path, 'rb') as f:
            tags = exifread.process_file(f)
            coords = get_coordinates(tags)
            if coords:
                coordinates.append(coords)

    if coordinates:
        centroid = [sum(x[0] for x in coordinates) / len(coordinates), sum(x[1] for x in coordinates) / len(coordinates)]
    else:
        centroid = [0, 0]

    map_obj = folium.Map(location=centroid)

    for image_path in image_paths:
        with open(image_path, 'rb') as f:
            tags = exifread.process_file(f)
            if tags:
                filename = os.path.basename(image_path)
                popup_text = f"Nombre de archivo: {filename}<br>"
                pkm_info = df[df['Nombre del Archivo'] == filename]
                if not pkm_info.empty:
                    distance = pkm_info['Distancia Acumulada (km)'].iloc[0]
                    integer_part = int(distance)
                    decimal_part = int((distance - integer_part) * 1000)
                    distance_formatted = f"{integer_part}+{decimal_part:03d}"
                    popup_text += f"PKM: {distance_formatted}<br>"
                add_image_marker(map_obj, image_path, popup_text)

    map_file = "mapa_con_imagenes.html"
    map_obj.save(map_file)

    # Leer el contenido del archivo HTML
    with open(map_file, 'r', encoding='utf-8') as f:
        map_html = f.read()

    # Mostrar el mapa usando iframe.srcdoc
    display(HTML(f'<iframe srcdoc="{map_html}" width="100%" height="600px"></iframe>'))

folder_path = "/content/imagenes_con_telemetria"
excel_path = "/content/informacion_imagenes.xlsx"
df = pd.read_excel(excel_path)
display_images_on_map(folder_path, df)



/usr/local/lib/python3.10/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


# **8.CREANDO UN KMZ CON IMAGENES EMBEBIDAS**

In [ ]:
import os
import base64
import exifread
import zipfile
import pandas as pd
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
from google.colab import files

def extract_image_info(image_path):
    try:
        with open(image_path, 'rb') as f:
            tags = exifread.process_file(f)
            if not tags:
                return None, None, None
            latitude = tags.get('GPS GPSLatitude')
            longitude = tags.get('GPS GPSLongitude')
            image_name = os.path.basename(image_path)

            if latitude and longitude:
                lat_degrees = latitude.values[0].num / latitude.values[0].den
                lat_minutes = latitude.values[1].num / latitude.values[1].den
                lat_seconds = latitude.values[2].num / latitude.values[2].den
                latitude_dec = lat_degrees + (lat_minutes / 60.0) + (lat_seconds / 3600.0)

                lon_degrees = longitude.values[0].num / longitude.values[0].den
                lon_minutes = longitude.values[1].num / longitude.values[1].den
                lon_seconds = longitude.values[2].num / longitude.values[2].den
                longitude_dec = lon_degrees + (lon_minutes / 60.0) + (lon_seconds / 3600.0)

                lat_dir = tags.get('GPS GPSLatitudeRef').printable
                lon_dir = tags.get('GPS GPSLongitudeRef').printable

                if lat_dir == 'S':
                    latitude_dec *= -1
                if lon_dir == 'W':
                    longitude_dec *= -1
            else:
                latitude_dec, longitude_dec = None, None

            return latitude_dec, longitude_dec, image_name
    except Exception as e:
        print(f"Error extracting info from {image_path}: {e}")
        return None, None, None

def compress_image(image_path, output_folder, max_size=(800, 600), quality=85):
    try:
        with Image.open(image_path) as img:
            img.thumbnail(max_size, Image.LANCZOS)
            compressed_path = os.path.join(output_folder, os.path.basename(image_path))
            img.save(compressed_path, format='JPEG', quality=quality)
        return compressed_path
    except Exception as e:
        print(f"Error compressing {image_path}: {e}")
        return None

def generate_kml_with_images(image_info_list, output_folder, df):
    placemark_template = """<Placemark>
    <name>{}</name>
    <description><![CDATA[<img src="data:image/jpeg;base64,{}" height="300" width="400"/>]]></description>
    <Point>
        <coordinates>{},{},0</coordinates>
    </Point>
</Placemark>
"""
    kml_content = """<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
<Document>
"""

    for latitude, longitude, image_name in image_info_list:
        if latitude is not None and longitude is not None:
            compressed_image_path = compress_image(os.path.join(folder_path, image_name), output_folder)
            if compressed_image_path:
                with open(compressed_image_path, 'rb') as f:
                    encoded_image = base64.b64encode(f.read()).decode('utf-8')
                # Obtener el texto de "Distancia Acumulada (km)" del DataFrame
                pkm_info = df[df['Nombre del Archivo'] == image_name]
                if not pkm_info.empty:
                    distance = pkm_info['Distancia Acumulada (km)'].iloc[0]
                    distance_str = f"{distance}"
                else:
                    distance_str = "Sin datos"
                kml_content += placemark_template.format(distance_str, encoded_image, longitude, latitude)
                os.remove(compressed_image_path)  # Remove the compressed image after use
            else:
                print(f"Failed to compress {image_name}")

    kml_content += """</Document>
</kml>"""

    return kml_content

def process_images(folder_path):
    image_info_list = []
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(extract_image_info, os.path.join(folder_path, filename))
                   for filename in os.listdir(folder_path)
                   if filename.lower().endswith(('.jpg', '.jpeg', '.png'))]
        for future in futures:
            result = future.result()
            if result is not None:
                image_info_list.append(result)
    return image_info_list

def generate_kmz(folder_path, kmz_path, df):
    output_folder = '/content/temp_images'
    os.makedirs(output_folder, exist_ok=True)

    print("Processing images...")
    image_info_list = process_images(folder_path)
    print("Generating KML content...")
    kml_content = generate_kml_with_images(image_info_list, output_folder, df)

    kml_file = os.path.join(output_folder, "images.kml")
    with open(kml_file, "w") as f:
        f.write(kml_content)

    print("Creating KMZ file...")
    with zipfile.ZipFile(kmz_path, 'w', zipfile.ZIP_DEFLATED) as kmz:
        kmz.write(kml_file, arcname=os.path.basename(kml_file))
    os.remove(kml_file)

    # Comprimir KMZ en un archivo ZIP
    zip_path = kmz_path.replace('.kmz', '.zip')
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(kmz_path, arcname=os.path.basename(kmz_path))
    os.remove(kmz_path)  # Eliminar el archivo KMZ para evitar duplicidad

    # Limpiar la carpeta temporal
    for file in os.listdir(output_folder):
        os.remove(os.path.join(output_folder, file))
    os.rmdir(output_folder)

    print("KMZ file created and compressed to ZIP.")
    return zip_path

folder_path = "/content/imagenes_con_telemetria"
kmz_path = "/content/images_KMZ.kmz"
excel_path = "/content/informacion_imagenes.xlsx"
df = pd.read_excel(excel_path)

zip_path = generate_kmz(folder_path, kmz_path, df)

# Verificar si el archivo ZIP ha sido creado y permitir su descarga
if os.path.exists(zip_path):
    print(f"Archivo ZIP creado: {zip_path}")
    files.download(zip_path)
else:
    print("Error: No se ha creado el archivo ZIP.")


Processing images...
Generating KML content...
Creating KMZ file...
KMZ file created and compressed to ZIP.
Archivo ZIP creado: /content/images_KMZ.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **ELIMINAR LA INFORMACION PARA EJECUTAR OTRO ANALISIS**

In [ ]:
!rm -rf /content/imagenes
!rm -rf /content/imagenes_con_telemetria
!rm -rf /content/imagenes_con_telemetria.zip
!rm -rf /content/images.kmz
!rm -rf /content/informacion_imagenes.xlsx
!rm -rf /content/mapa_con_imagenes.html

Fin.